In [1]:
from collections import Counter
from math import sqrt
from random import Random
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import re
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import svds
#from scipy.stats.stats import spearmanr
from scipy.stats import spearmanr
from random import shuffle
import math
import random
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
#from sparsesvd import sparsesvd
from numpy.linalg import matrix_rank
from scipy.stats import chi2_contingency
from matplotlib.pyplot import *
from numpy import inf
from scipy.linalg import svd
from scipy.linalg import eigh

from foobar import *

In [2]:
#load basic matrix and elements that are neeeded for later
cooccur_dense = np.load("cooccur_dense_thr10000.npy")
vocab = np.load("vocabthr10000_dic_index.npy", allow_pickle=True)
vocab = dict(enumerate(vocab.flatten(), 1))[1]

test_files = ['wordsim353.txt', 'men_dataset.txt', 'mturk.txt', 'rarewords.txt', 'simlex999.txt']

In [3]:
dim_range1 = [2, 50]
dim_range2 = list(np.arange(100, 1001, 100))
dim_range3 = list(np.arange(2000, 10001, 1000))
dim_range = [*dim_range1, *dim_range2, *dim_range3]
print(dim_range)
eig_weight = [0, 0.5]
eig_weight_str = ['0', '0.5']
rank = 15135

[2, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]


In [4]:
#To look at word pairs in Wiki052024

cooccur_vectors_words = fittingmatrixvectors(cooccur_dense)

test_word_pairs = []

for i in range(len(test_files)):
    
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    
    test_word_pairs_set = read_test_word_pairs_set(path_input)
    
    evaluate_word_pairs_create = evaluate_word_pairs(cooccur_vectors_words, test_word_pairs_set, vocab, "cosine")
    
    test_word_pairs.append((len(test_word_pairs_set), evaluate_word_pairs_create[1], float(evaluate_word_pairs_create[1]) / len(test_word_pairs_set)))
#table 1
test_word_pairs

[(353, 294, 0.8328611898016998),
 (3000, 2159, 0.7196666666666667),
 (287, 256, 0.89198606271777),
 (2034, 294, 0.14454277286135694),
 (999, 810, 0.8108108108108109)]

In [5]:
#Evaluate TTEST, PMI, and PPMI

ttest = build_ttest(cooccur_dense)
np.save("decomposed matrix\\thr10000_ttest.npy", ttest)
ttest_vectors_words = fittingmatrixvectors(ttest)
ws_score_ttest = []
for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_ttest.append(("ttest", test_files[i], ws_evaluate(ttest_vectors_words, vocab, mearsure = 'cosine', path = path_input)))      
np.save("table2\\thr10000_ws_score_ttest.npy", ws_score_ttest)  
del ttest
del ttest_vectors_words
del ws_score_ttest

MemoryError: Unable to allocate 1.71 GiB for an array with shape (15135, 15135) and data type float64

In [3]:
pmi = build_count_transform(cooccur_dense, contratype = "pmi")
np.save("decomposed matrix\\thr10000_pmi.npy", pmi)

pmi_vectors_words = fittingmatrixvectors(pmi)
ws_score_pmi = []
for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_pmi.append(("pmi", test_files[i], ws_evaluate(pmi_vectors_words, vocab, mearsure = 'cosine', path = path_input)))  
np.save("table2\\thr10000_ws_score_pmi.npy", ws_score_pmi)  

del pmi
del pmi_vectors_words

D:\20240901paper3\20240920\foobar.py:59: RuntimeWarning: divide by zero encountered in log2
  pmi = np.log2(pmi)


In [5]:
del ws_score_pmi

ppmi = build_count_transform(cooccur_dense, contratype = "ppmi")
np.save("decomposed matrix\\thr10000_ppmi.npy", ppmi)

ppmi_vectors_words = fittingmatrixvectors(ppmi)
ws_score_ppmi = []

for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_ppmi.append(("ppmi", test_files[i], ws_evaluate(ppmi_vectors_words, vocab, mearsure = 'cosine', path = path_input)))    
    
np.save("table2\\thr10000_ws_score_ppmi.npy", ws_score_ppmi)  

D:\20240901paper3\20240920\foobar.py:69: RuntimeWarning: divide by zero encountered in log2
  pmi = np.log2(pmi)


In [6]:
del ppmi
del ppmi_vectors_words

In [8]:
del ws_score_ppmi

In [7]:
ws_score_ttest

[('ttest', 'wordsim353.txt', 0.5154054998682059),
 ('ttest', 'men_dataset.txt', 0.20805313141861317),
 ('ttest', 'mturk.txt', 0.5672027665698709),
 ('ttest', 'rarewords.txt', 0.43564018467701815),
 ('ttest', 'simlex999.txt', 0.25087934747538454)]

In [4]:
ws_score_pmi

[('pmi', 'wordsim353.txt', 0.22376287259551497),
 ('pmi', 'men_dataset.txt', 0.1373063844621597),
 ('pmi', 'mturk.txt', 0.4135604750664361),
 ('pmi', 'rarewords.txt', 0.25623975048136605),
 ('pmi', 'simlex999.txt', 0.18399052693539292)]

In [7]:
ws_score_ppmi

[('ppmi', 'wordsim353.txt', 0.6061350468104976),
 ('ppmi', 'men_dataset.txt', 0.2623660732940164),
 ('ppmi', 'mturk.txt', 0.636315760559693),
 ('ppmi', 'rarewords.txt', 0.47935612595199617),
 ('ppmi', 'simlex999.txt', 0.3522010605252525)]

In [9]:
decomposedmatrix = ["root", "rootroot"]
#create the word-context matrix of PMI, PPMI, ROOT, ROOTROOT

for i in range(len(decomposedmatrix)):
    temp = build_count_transform(cooccur_dense, contratype = decomposedmatrix[i])
    np.save("decomposed matrix\\thr10000_"+decomposedmatrix[i]+".npy", temp)
#copy cooccur_dense and change the name into text8_raw

In [10]:
del temp

In [11]:
thr10000_root = np.load("decomposed matrix\\thr10000_root.npy")
thr10000_root_ttest = build_ttest(thr10000_root)
np.save("decomposed matrix\\thr10000_root_ttest.npy", thr10000_root_ttest)
del thr10000_root
del thr10000_root_ttest

thr10000_rootroot = np.load("decomposed matrix\\thr10000_rootroot.npy")
thr10000_rootroot_ttest = build_ttest(thr10000_rootroot)
np.save("decomposed matrix\\thr10000_rootroot_ttest.npy", thr10000_rootroot_ttest)
del thr10000_rootroot
del thr10000_rootroot_ttest

#create the word-context matrix of STRATOS-TTEST

thr10000_rootcca = build_count_transform(cooccur_dense, contratype = "rootcca")
np.save("decomposed matrix\\thr10000_rootcca.npy", thr10000_rootcca)
del thr10000_rootcca

thr10000_pmi = np.load("decomposed matrix\\thr10000_pmi.npy")


beta = 1
a = 1
b = 1
P = cooccur_dense / np.sum(cooccur_dense)
sum_w = np.array(P.sum(axis = 1))
sum_c = np.array(P.sum(axis=0)).T
sum_c_beta = (sum_c ** (beta)) / np.sum(sum_c ** (beta)) 
sum_w_pro_root = sum_w ** (a/2)
sum_c_pro_root = sum_c_beta ** (b/2)
        
sum_w_pro_root_diag = np.diag(sum_w_pro_root)
sum_c_pro_root_diag = np.diag(sum_c_pro_root)
  
thr10000_pmi_gsvd = sum_w_pro_root_diag @ thr10000_pmi @ sum_c_pro_root_diag
np.save("decomposed matrix\\thr10000_pmi_gsvd.npy", thr10000_pmi_gsvd)


In [12]:
del beta
del a
del b
del P
del sum_w
del sum_c
del sum_c_beta
del sum_w_pro_root
del sum_c_pro_root
del sum_w_pro_root_diag
del sum_c_pro_root_diag
del thr10000_pmi_gsvd

In [13]:
#Evaluate ROOT-TTEST and save the evaluation
root_ttest = np.load("decomposed matrix\\thr10000_root_ttest.npy")
root_ttest_vectors_words = fittingmatrixvectors(root_ttest)
ws_score_root_ttest = []
for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_root_ttest.append(("root_ttest", test_files[i], ws_evaluate(root_ttest_vectors_words, vocab, mearsure = 'cosine', path = path_input)))      
np.save("table2\\thr10000_ws_score_root_ttest.npy", ws_score_root_ttest)  
print(ws_score_root_ttest)
#Evaluate ROOTROOT-TTEST and save the evaluation
del root_ttest
del root_ttest_vectors_words

print(ws_score_root_ttest)

del ws_score_root_ttest

rootroot_ttest = np.load("decomposed matrix\\thr10000_rootroot_ttest.npy")
rootroot_ttest_vectors_words = fittingmatrixvectors(rootroot_ttest)
ws_score_rootroot_ttest = []
for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_rootroot_ttest.append(("rootroot_ttest", test_files[i], ws_evaluate(rootroot_ttest_vectors_words, vocab, mearsure = 'cosine', path = path_input)))      
np.save("table2\\thr10000_ws_score_rootroot_ttest.npy", ws_score_rootroot_ttest)  
del rootroot_ttest
del rootroot_ttest_vectors_words
print(ws_score_rootroot_ttest)
del ws_score_rootroot_ttest

rootcca = np.load("decomposed matrix\\thr10000_rootcca.npy")
rootcca_vectors_words = fittingmatrixvectors(rootcca)
ws_score_rootcca = []
for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_rootcca.append(("rootcca", test_files[i], ws_evaluate(rootcca_vectors_words, vocab, mearsure = 'cosine', path = path_input)))      
np.save("table2\\thr10000_ws_score_rootcca.npy", ws_score_rootcca)  
del rootcca
del rootcca_vectors_words

print(ws_score_rootcca)
del ws_score_rootcca

pmi_gsvd = np.load("decomposed matrix\\thr10000_pmi_gsvd.npy")

pmi_g_vectors_words = fittingmatrixvectors(pmi_gsvd)
ws_score_pmi_g = []

for i in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    ws_score_pmi_g.append(("pmi_g", test_files[i], ws_evaluate(pmi_g_vectors_words, vocab, mearsure = 'cosine', path = path_input)))      
    
np.save("table2\\thr10000_ws_score_pmi_g.npy", ws_score_pmi_g)  
del pmi_gsvd
del pmi_g_vectors_words
print(ws_score_pmi_g)
del ws_score_pmi_g

[('root_ttest', 'wordsim353.txt', 0.5881215979327212), ('root_ttest', 'men_dataset.txt', 0.2514605903730615), ('root_ttest', 'mturk.txt', 0.6572925055943496), ('root_ttest', 'rarewords.txt', 0.4966643225577744), ('root_ttest', 'simlex999.txt', 0.34252519890148464)]
[('root_ttest', 'wordsim353.txt', 0.5881215979327212), ('root_ttest', 'men_dataset.txt', 0.2514605903730615), ('root_ttest', 'mturk.txt', 0.6572925055943496), ('root_ttest', 'rarewords.txt', 0.4966643225577744), ('root_ttest', 'simlex999.txt', 0.34252519890148464)]
[('rootroot_ttest', 'wordsim353.txt', 0.5067348851208608), ('rootroot_ttest', 'men_dataset.txt', 0.2358313083585509), ('rootroot_ttest', 'mturk.txt', 0.619517483279037), ('rootroot_ttest', 'rarewords.txt', 0.4261663785322584), ('rootroot_ttest', 'simlex999.txt', 0.3035801288086941)]
[('rootcca', 'wordsim353.txt', 0.4381680358160836), ('rootcca', 'men_dataset.txt', 0.14091223186275023), ('rootcca', 'mturk.txt', 0.5508197251928694), ('rootcca', 'rarewords.txt', 0.33

In [14]:
#Create and evaluate PMI-SVD, PPMI-SVD, and RAW-CA, ROOT-CA, ROOTROOT-CA

svddecomposedmatrix = ["pmi", "ppmi"]
for i in range(len(svddecomposedmatrix)):
    print(i)
    temp = np.load("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+".npy")
    svd_u, svd_s, svd_v = svdcadense(temp, method = "svd")
    np.save("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+"_svd_u.npy", svd_u)
    np.save("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+"_svd_s.npy", svd_s)
    np.save("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+"_svd_v.npy", svd_v)

0
1


In [15]:
del temp
del svd_u
del svd_s
del svd_v

In [17]:
del cooccur_dense

In [5]:
cadecomposedmatrix = ["raw", "root", "rootroot"]
for i in range(len(cadecomposedmatrix)):
    print(i)
    temp = np.load("decomposed matrix\\thr10000_"+cadecomposedmatrix[i]+"_ttest.npy")
    ca_u, ca_s, ca_v = svdcadense(temp, method = "svd")
    np.save("decomposed matrix\\thr10000_"+cadecomposedmatrix[i]+"_ca_u.npy", ca_u)
    np.save("decomposed matrix\\thr10000_"+cadecomposedmatrix[i]+"_ca_s.npy", ca_s)
    np.save("decomposed matrix\\thr10000_"+cadecomposedmatrix[i]+"_ca_v.npy", ca_v)

0
1
2


In [6]:
#create and evaluate ROOT-CCA

svddecomposedmatrix = ["rootcca"]
for i in range(len(svddecomposedmatrix)):
    print(i)
    temp = np.load("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+".npy")
    svd_u, svd_s, svd_v = svdcadense(temp, method = "svd")
    np.save("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+"_svd_u.npy", svd_u)
    np.save("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+"_svd_s.npy", svd_s)
    np.save("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+"_svd_v.npy", svd_v)

0


In [7]:
#Create and evaluate PMI-GSVD

svddecomposedmatrix = ["pmi_gsvd"]
for i in range(len(svddecomposedmatrix)):
    print(i)
    temp = np.load("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+".npy")
    gsvd_u, gsvd_s, gsvd_v = svdcadense(temp, method = "svd")
    np.save("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+"_u.npy", gsvd_u)
    np.save("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+"_s.npy", gsvd_s)
    np.save("decomposed matrix\\thr10000_"+svddecomposedmatrix[i]+"_v.npy", gsvd_v)

0


In [8]:
svddecomposedmatrix = ["pmi", "ppmi"]

for q in range(len(svddecomposedmatrix)):
    print(q)
    svd_u = np.load("decomposed matrix\\thr10000_"+svddecomposedmatrix[q]+"_svd_u.npy")
    svd_s = np.load("decomposed matrix\\thr10000_"+svddecomposedmatrix[q]+"_svd_s.npy")
    svd_ws_score_cos_all = list()
    for k in range(len(test_files)):
        print("k = " + str(test_files[k]))
        test_type = test_files[k]
    
        path_input = "similarities\\"+test_files[k]

        svd_ws_score_cos = pd.DataFrame()
    
        for i in range(len(eig_weight)):
            print("i = " + str(eig_weight[i]))
    
            svd_w = svd_u * np.power(svd_s, eig_weight[i])

            svd_score = []

            for j in dim_range:
                vectors_words = svdvectorsincrease(svd_w, dim = j)
        
                svd_score_dim = ws_evaluate(vectors_words, vocab, mearsure = 'cosine', path = path_input)
            
                svd_score.append(svd_score_dim)

            svd_ws_score_cos[eig_weight_str[i]] = svd_score
        svd_ws_score_cos_all.append(svd_ws_score_cos)
    
    np.save("thr10000_"+svddecomposedmatrix[q]+"_svd_ws_score_cos_all.npy", svd_ws_score_cos_all)

    
cadecomposedmatrix = ["raw", "root", "rootroot"]

for q in range(len(cadecomposedmatrix)):
    print(q)
    ca_u = np.load("decomposed matrix\\thr10000_"+cadecomposedmatrix[q]+"_ca_u.npy")
    ca_s = np.load("decomposed matrix\\thr10000_"+cadecomposedmatrix[q]+"_ca_s.npy")
    ca_ws_score_cos_all = list()
    for k in range(len(test_files)):
        print("k = " + str(test_files[k]))
        test_type = test_files[k]
    
        path_input = "similarities\\"+test_files[k]

        ca_ws_score_cos = pd.DataFrame()
    
        for i in range(len(eig_weight)):
            print("i = " + str(eig_weight[i]))
    
            ca_w = ca_u * np.power(ca_s, eig_weight[i])

            ca_score = []

            for j in dim_range:
                vectors_words = svdvectorsincrease(ca_w, dim = j)
        
                ca_score_dim = ws_evaluate(vectors_words, vocab, mearsure = 'cosine', path = path_input)
            
                ca_score.append(ca_score_dim)

            ca_ws_score_cos[eig_weight_str[i]] = ca_score
        ca_ws_score_cos_all.append(ca_ws_score_cos)
    
    np.save("thr10000_"+cadecomposedmatrix[q]+"_ca_ws_score_cos_all.npy", ca_ws_score_cos_all)    
    
    
svddecomposedmatrix = ["rootcca"]

for q in range(len(svddecomposedmatrix)):
    print(q)
    svd_u = np.load("decomposed matrix\\thr10000_"+svddecomposedmatrix[q]+"_svd_u.npy")
    svd_s = np.load("decomposed matrix\\thr10000_"+svddecomposedmatrix[q]+"_svd_s.npy")
    svd_ws_score_cos_all = list()
    for k in range(len(test_files)):
        print("k = " + str(test_files[k]))
        test_type = test_files[k]
    
        path_input = "similarities\\"+test_files[k]

        svd_ws_score_cos = pd.DataFrame()
    
        for i in range(len(eig_weight)):
            print("i = " + str(eig_weight[i]))
    
            svd_w = svd_u * np.power(svd_s, eig_weight[i])

            svd_score = []

            for j in dim_range:
                vectors_words = svdvectorsincrease(svd_w, dim = j)
        
                svd_score_dim = ws_evaluate(vectors_words, vocab, mearsure = 'cosine', path = path_input)
            
                svd_score.append(svd_score_dim)

            svd_ws_score_cos[eig_weight_str[i]] = svd_score
        svd_ws_score_cos_all.append(svd_ws_score_cos)
    
    np.save("thr10000_"+svddecomposedmatrix[q]+"_svd_ws_score_cos_all.npy", svd_ws_score_cos_all)
    
svddecomposedmatrix = ["pmi"]

for q in range(len(svddecomposedmatrix)):
    print(q)
    svd_u = np.load("decomposed matrix\\thr10000_"+svddecomposedmatrix[q]+"_gsvd_u.npy")
    svd_s = np.load("decomposed matrix\\thr10000_"+svddecomposedmatrix[q]+"_gsvd_s.npy")
    gsvd_ws_score_cos_all = list()
    for k in range(len(test_files)):
        print("k = " + str(test_files[k]))
        test_type = test_files[k]
    
        path_input = "similarities\\"+test_files[k]

        svd_ws_score_cos = pd.DataFrame()
    
        for i in range(len(eig_weight)):
            print("i = " + str(eig_weight[i]))
    
            svd_w = svd_u * np.power(svd_s, eig_weight[i])

            svd_score = []

            for j in dim_range:
                vectors_words = svdvectorsincrease(svd_w, dim = j)
        
                svd_score_dim = ws_evaluate(vectors_words, vocab, mearsure = 'cosine', path = path_input)
            
                svd_score.append(svd_score_dim)

            svd_ws_score_cos[eig_weight_str[i]] = svd_score
        gsvd_ws_score_cos_all.append(svd_ws_score_cos)
    
    np.save("thr10000_"+svddecomposedmatrix[q]+"_gsvd_ws_score_cos_all.npy", gsvd_ws_score_cos_all)

0
k = wordsim353.txt
i = 0
i = 0.5
k = men_dataset.txt
i = 0
i = 0.5
k = mturk.txt
i = 0
i = 0.5
k = rarewords.txt
i = 0
i = 0.5
k = simlex999.txt
i = 0
i = 0.5
1
k = wordsim353.txt
i = 0
i = 0.5
k = men_dataset.txt
i = 0
i = 0.5
k = mturk.txt
i = 0
i = 0.5
k = rarewords.txt
i = 0
i = 0.5
k = simlex999.txt
i = 0
i = 0.5
0
k = wordsim353.txt
i = 0
i = 0.5
k = men_dataset.txt
i = 0
i = 0.5
k = mturk.txt
i = 0
i = 0.5
k = rarewords.txt
i = 0
i = 0.5
k = simlex999.txt
i = 0
i = 0.5
1
k = wordsim353.txt
i = 0
i = 0.5
k = men_dataset.txt
i = 0
i = 0.5
k = mturk.txt
i = 0
i = 0.5
k = rarewords.txt
i = 0
i = 0.5
k = simlex999.txt
i = 0
i = 0.5
2
k = wordsim353.txt
i = 0
i = 0.5
k = men_dataset.txt
i = 0
i = 0.5
k = mturk.txt
i = 0
i = 0.5
k = rarewords.txt
i = 0
i = 0.5
k = simlex999.txt
i = 0
i = 0.5
0
k = wordsim353.txt
i = 0
i = 0.5
k = men_dataset.txt
i = 0
i = 0.5
k = mturk.txt
i = 0
i = 0.5
k = rarewords.txt
i = 0
i = 0.5
k = simlex999.txt
i = 0
i = 0.5
0
k = wordsim353.txt
i = 0
i = 0.5